# NOTE:  THIS NOTEBOOK WILL TAKE ABOUT 30 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

# Fine-Tuning a Generative Model 

In the previous section, we've already performed the Feature Engineering to create embeddings from the `reviews_body` text using a pre-trained model.  We  split the dataset into train, validation and test files. To optimize for fine-tuning training, we saved the files in Arrow format. 

Now, let’s fine-tune the model with the `review_body` data Amazon Customer Reviews Dataset.

![Generative fine-tuning](img/bert_training.png)

In [2]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region, 
                            config=config)

# _PRE-REQUISITE: You need to have succesfully run the notebooks in the `PREPARE` section before you continue with this notebook._

In [3]:
%store -r processed_train_data_s3_uri

In [4]:
try:
    processed_train_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(processed_train_data_s3_uri)

s3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/train


In [6]:
%store -r processed_validation_data_s3_uri

In [7]:
try:
    processed_validation_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
print(processed_validation_data_s3_uri)

s3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/validation


In [9]:
%store -r processed_test_data_s3_uri

In [10]:
try:
    processed_test_data_s3_uri
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [11]:
print(processed_test_data_s3_uri)

s3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/test


In [12]:
%store -r model_checkpoint

In [13]:
try:
    model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [14]:
print(model_checkpoint)

bigscience/bloomz-560m


In [15]:
%store -r dataset_templates_name

In [16]:
try:
    dataset_templates_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [17]:
print(dataset_templates_name)

amazon_us_reviews/Wireless_v1_00


In [18]:
%store -r prompt_template_name

In [19]:
try:
    prompt_template_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [20]:
print(prompt_template_name)

Given the review body return a categorical rating


# Specify the Dataset in S3
We are using the train, validation, and test splits created in the previous section.

In [21]:
print(processed_train_data_s3_uri)

!aws s3 ls $processed_train_data_s3_uri/

s3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/train
2023-03-10 01:52:57   12095208 amazon_reviews_us_Digital_Software_v1_00.parquet
2023-03-10 01:53:12   13611143 amazon_reviews_us_Digital_Video_Games_v1_00.parquet
2023-03-10 01:52:57    1438759 amazon_reviews_us_Gift_Card_v1_00.parquet


In [22]:
print(processed_validation_data_s3_uri)

!aws s3 ls $processed_validation_data_s3_uri/

s3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/validation
2023-03-10 01:52:58     687294 amazon_reviews_us_Digital_Software_v1_00.parquet
2023-03-10 01:53:13     773591 amazon_reviews_us_Digital_Video_Games_v1_00.parquet
2023-03-10 01:52:58      79277 amazon_reviews_us_Gift_Card_v1_00.parquet


In [23]:
print(processed_test_data_s3_uri)

!aws s3 ls $processed_test_data_s3_uri/

s3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/test
2023-03-10 01:52:58     687294 amazon_reviews_us_Digital_Software_v1_00.parquet
2023-03-10 01:53:13     773591 amazon_reviews_us_Digital_Video_Games_v1_00.parquet
2023-03-10 01:52:58      79277 amazon_reviews_us_Gift_Card_v1_00.parquet


# Specify S3 `Distribution Strategy`

In [24]:
!aws s3 ls --recursive s3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-00-31-03-169

2023-03-10 00:31:04      11976 sagemaker-scikit-learn-2023-03-10-00-31-03-169/input/code/preprocess.py


In [25]:
from sagemaker.inputs import TrainingInput

s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri, distribution="ShardedByS3Key")
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri, distribution="ShardedByS3Key")
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri, distribution="ShardedByS3Key")

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-44-04-130/output/test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


# Setup Hyper-Parameters for Classification Layer

In [26]:
print(model_checkpoint)

bigscience/bloomz-560m


In [27]:
epochs = 1
learning_rate = 0.00001
train_batch_size = 128
validation_batch_size = 128
test_batch_size = 128
train_steps_per_epoch = 10
validation_steps = 10
test_steps = 10
train_instance_count = 1
train_instance_type = "ml.c5.9xlarge"
train_volume_size = 1024
#use_xla = True
#use_amp = True
#freeze_bert_layer = False
enable_sagemaker_debugger = False
enable_checkpointing = False
enable_tensorboard = False
input_mode = "FastFile"
run_validation = False
run_test = False
run_sample_predictions = False

# Setup Metrics To Track Model Performance

These sample log lines...
```
45/50 [=====>..] - ETA: 3s - loss: 0.425 - accuracy: 0.881
50/50 [=======>] - ETA: 0s - val_loss: 0.407 - val_accuracy: 0.885
```
...will produce the following 4 metrics in CloudWatch:

`loss` = 0.425

`accuracy` = 0.881

`val_loss` = 0.407

`val_accuracy` = 0.885

<img src="img/cloudwatch_train_metrics.png" align="left">

In [28]:
metrics_definitions = [
    {"Name": "train:loss", "Regex": "loss: ([0-9\\.]+)"},
    {"Name": "train:accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
    {"Name": "validation:loss", "Regex": "val_loss: ([0-9\\.]+)"},
    {"Name": "validation:accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
]

# Setup SageMaker Debugger
Define Debugger Rules as deccribed here:  https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html

In [29]:
from sagemaker.debugger import Rule
from sagemaker.debugger import rule_configs
from sagemaker.debugger import ProfilerRule
from sagemaker.debugger import CollectionConfig
from sagemaker.debugger import DebuggerHookConfig

actions = rule_configs.ActionList(
    #    rule_configs.StopTraining(),
    #    rule_configs.Email("")
)

rules = [
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),    
#     ProfilerRule.sagemaker(rule_configs.BatchSize()),
#     ProfilerRule.sagemaker(rule_configs.CPUBottleneck()),
#     ProfilerRule.sagemaker(rule_configs.GPUMemoryIncrease()),
#     ProfilerRule.sagemaker(rule_configs.IOBottleneck()),
#     ProfilerRule.sagemaker(rule_configs.LoadBalancing()),
#     ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
#     ProfilerRule.sagemaker(rule_configs.OverallSystemUsage()),
#     ProfilerRule.sagemaker(rule_configs.StepOutlier()),
#     Rule.sagemaker(
#         base_config=rule_configs.loss_not_decreasing(),
#         rule_parameters={
#             "collection_names": "losses,metrics",
#             "use_losses_collection": "true",
#             "num_steps": "10",
#             "diff_percent": "50",
#         },
#         collections_to_save=[
#             CollectionConfig(
#                 name="losses",
#                 parameters={
#                     "save_interval": "10",
#                 },
#             ),
#             CollectionConfig(
#                 name="metrics",
#                 parameters={
#                     "save_interval": "10",
#                 },
#             ),
#         ],
#         actions=actions,
#     ),
#     Rule.sagemaker(
#         base_config=rule_configs.overtraining(),
#         rule_parameters={
#             "collection_names": "losses,metrics",
#             "patience_train": "10",
#             "patience_validation": "10",
#             "delta": "0.5",
#         },
#         collections_to_save=[
#             CollectionConfig(
#                 name="losses",
#                 parameters={
#                     "save_interval": "10",
#                 },
#             ),
#             CollectionConfig(
#                 name="metrics",
#                 parameters={
#                     "save_interval": "10",
#                 },
#             ),
#         ],
#         actions=actions,
#     )    
]

hook_config = DebuggerHookConfig(
    hook_parameters={
        "save_interval": "10",  # number of steps
        "export_tensorboard": "true",
        "tensorboard_dir": "hook_tensorboard/",
    }
)

## Specify a Debugger profiler configuration

The following configuration will capture system metrics at 500 milliseconds. The system metrics include utilization per CPU, GPU, memory utilization per CPU, GPU as well I/O and network.

Debugger will capture detailed profiling information from step 5 to step 15. This information includes Horovod metrics, dataloading, preprocessing, operators running on CPU and GPU.

In [30]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(local_path="/opt/ml/output/profiler/", start_step=5, num_steps=10),
)

# Specify Checkpoint S3 Location
This is used for Spot Instances Training.  If nodes are replaced, the new node will start training from the latest checkpoint.

In [31]:
import uuid

checkpoint_s3_prefix = "checkpoints/{}".format(str(uuid.uuid4()))
checkpoint_s3_uri = "s3://{}/{}/".format(bucket, checkpoint_s3_prefix)

print(checkpoint_s3_uri)

s3://sagemaker-us-east-1-079002598131/checkpoints/6133d025-3902-4893-9644-f3c760d4e15a/


# Setup Our BERT + TensorFlow Script to Run on SageMaker
Prepare our TensorFlow model to run on the managed SageMaker service

In [32]:
!pygmentize src/train.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os
import csv

import pandas as pd
import numpy as np

subprocess.check_call([sys.executable, "-m", "pip", "install", "torch==1.13.1", "torchdata==0.5.1"])
import torch

subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers==4.26.1", "datasets==2.9.0"])
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM


# def load_checkpoint_model(checkpoint_path):
#     import glob
#     import os

#     glob_pattern = os.path.join(checkpoint_path, "*.h5")
#     print("glob pattern {}".format(glob_pattern))

#     list_of_checkpoint_files = glob.glob(glob_pattern)
#     print("List of checkpoint files {}".format(list_of_checkpoint_files))

#     latest_checkpoint_file = max(list_of_checkpoint_files)
#     print("Latest checkpoint file {}".format(latest_checkpoint_file))

#     initial

In [33]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="train.py",
    source_dir="src",
    role=role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=train_volume_size,
    checkpoint_s3_uri=checkpoint_s3_uri,
    py_version="py39",
    framework_version="1.13",
    hyperparameters={
        "epochs": epochs,
        "learning_rate": learning_rate,
        # "epsilon": epsilon,
        "train_batch_size": train_batch_size,
        "validation_batch_size": validation_batch_size,
        "test_batch_size": test_batch_size,
        "train_steps_per_epoch": train_steps_per_epoch,
        "validation_steps": validation_steps,
        "test_steps": test_steps,
        # "use_xla": use_xla,
        # "use_amp": use_amp,
        "model_checkpoint": model_checkpoint,
        "dataset_templates_name": dataset_templates_name,
        "prompt_template_name": prompt_template_name,
        "enable_checkpointing": enable_checkpointing,
        "enable_tensorboard": enable_tensorboard,
        "run_validation": run_validation,
        "run_test": run_test,
        "run_sample_predictions": run_sample_predictions,
    },
    input_mode=input_mode,
    metric_definitions=metrics_definitions,
    rules=rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

# Train the Model on SageMaker

In [34]:
!aws s3 ls --recursive s3://sagemaker-us-east-1-079002598131/sagemaker-scikit-learn-2023-03-10-01-17-28-178/

2023-03-10 01:17:29      11976 sagemaker-scikit-learn-2023-03-10-01-17-28-178/input/code/preprocess.py


In [35]:
estimator.fit(
    inputs={"train": s3_input_train_data, "validation": s3_input_validation_data, "test": s3_input_test_data},
    wait=False,
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-03-10-02-07-14-667


In [36]:
training_job_name = estimator.latest_training_job.name
print("Training Job Name:  {}".format(training_job_name))

Training Job Name:  pytorch-training-2023-03-10-02-07-14-667


In [37]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

In [38]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

In [39]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(
            bucket, training_job_name, region
        )
    )
)

In [40]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Checkpoint Data</a> After The Training Job Has Completed</b>'.format(
            bucket, checkpoint_s3_prefix, region
        )
    )
)

In [41]:
%%time

estimator.latest_training_job.wait(logs=False)


2023-03-10 02:07:16 Starting - Starting the training job...
2023-03-10 02:07:31 Starting - Preparing the instances for training........
2023-03-10 02:08:22 Downloading - Downloading input data....
2023-03-10 02:08:42 Training - Downloading the training image............
2023-03-10 02:09:48 Training - Training image download completed. Training in progress...........................
2023-03-10 02:12:09 Uploading - Uploading generated training model...........................................................
2023-03-10 02:17:10 Completed - Training job completed
CPU times: user 554 ms, sys: 57.5 ms, total: 612 ms
Wall time: 9min 56s


# Wait Until the ^^ Training Job ^^ Completes Above!

# Display Training Job Metrics

In [42]:
estimator.training_job_analytics.dataframe()

""


# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

In [43]:
%store training_job_name

Stored 'training_job_name' (str)


In [44]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-079002598131/pytorch-training-2023-03-10-02-07-14-667/output/model.tar.gz to ./model.tar.gz


In [45]:
!mkdir -p ./model/
!tar -xvzf ./model.tar.gz -C ./model/

code/
code/inference.py
code/requirements.txt
config.json
tensorboard/
special_tokens_map.json
generation_config.json
test_data/
test_data/gpt3-test/
test_data/gpt3-test/amazon_reviews_us_Gift_Card_v1_00.parquet
test_data/gpt3-test/amazon_reviews_us_Digital_Software_v1_00.parquet
test_data/gpt3-test/amazon_reviews_us_Digital_Video_Games_v1_00.parquet
tokenizer.json
tokenizer_config.json
pytorch_model.bin


# Analyze Debugger Rules

In [48]:
estimator.latest_training_job.rule_job_summary()

[{'RuleConfigurationName': 'ProfilerReport',
  'RuleEvaluationJobArn': 'arn:aws:sagemaker:us-east-1:079002598131:processing-job/pytorch-training-2023-03-1-profilerreport-72575b59',
  'RuleEvaluationStatus': 'NoIssuesFound',
  'LastModifiedTime': datetime.datetime(2023, 3, 10, 2, 17, 25, 223000, tzinfo=tzlocal())}]

In [49]:
%store

Stored variables and their in-db values:
balance_dataset                                       -> True
dataset_templates_name                                -> 'amazon_us_reviews/Wireless_v1_00'
ingest_create_athena_table_parquet_passed             -> True
model_checkpoint                                      -> 'bigscience/bloomz-560m'
pipeline_endpoint_name                                -> 'model-from-registry-ep-1678382518'
pipeline_experiment_name                              -> 'pipeline-1678413961'
pipeline_name                                         -> 'pipeline-1678413961'
pipeline_trial_name                                   -> 'trial-1678413962'
processed_test_data_s3_uri                            -> 's3://sagemaker-us-east-1-079002598131/sagemaker-s
processed_train_data_s3_uri                           -> 's3://sagemaker-us-east-1-079002598131/sagemaker-s
processed_validation_data_s3_uri                      -> 's3://sagemaker-us-east-1-079002598131/sagemaker-s
prompt_tem

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>